In [2]:
import numpy as np
import pandas as pd
import json

In [3]:
def read_data(path):
    '''
    this function will read the txt file which is a line-delimited json file and produce a pandas dataframe.
    
    '''
    my_list =[]
    with open(path) as f:
        for line in f:
            json_content = json.loads(line)
            my_list.append(json_content)
    #df = pd.DataFrame(my_list) 
    return(pd.DataFrame(my_list))
    

In [4]:
df = read_data('transactions.txt')

In [5]:
df.head()

,accountNumber,accountOpenDate,acqCountry,availableMoney,cardCVV,cardLast4Digits,cardPresent,creditLimit,currentBalance,currentExpDate,...,merchantName,merchantState,merchantZip,posConditionCode,posEntryMode,posOnPremises,recurringAuthInd,transactionAmount,transactionDateTime,transactionType
0,733493772,2014-08-03,US,5000.00,492,9184,False,5000.0,0.00,04/2020,...,Lyft,,,01,05,,,111.33,2016-01-08T19:04:50,PURCHASE
1,733493772,2014-08-03,US,4888.67,492,9184,False,5000.0,111.33,06/2023,...,Uber,,,01,09,,,24.75,2016-01-09T22:32:39,PURCHASE
2,733493772,2014-08-03,US,4863.92,492,9184,False,5000.0,136.08,12/2027,...,Lyft,,,01,05,,,187.40,2016-01-11T13:36:55,PURCHASE
3,733493772,2014-08-03,US,4676.52,492,9184,False,5000.0,323.48,09/2029,...,Lyft,,,01,02,,,227.34,2016-01-11T22:47:46,PURCHASE
4,733493772,2014-08-03,US,4449.18,492,9184,False,5000.0,550.82,10/2024,...,Lyft,,,01,02,,,0.00,2016-01-16T01:41:11,ADDRESS_VERIFICATION


In [6]:
df.shape

(641914, 29)

In [7]:
df.replace(r'^\s*$', np.nan, regex=True, inplace=True) # Replacing blank values with nan. 

In [8]:
df.info() # this would give us a quick description of the data, in particular the total number of observations, and each attribute's type and number of non-null values.
# There are 641914 instances in the dataset. We have some numerical attributes like 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 641914 entries, 0 to 641913
Data columns (total 29 columns):
accountNumber               641914 non-null object
accountOpenDate             641914 non-null object
acqCountry                  638001 non-null object
availableMoney              641914 non-null float64
cardCVV                     641914 non-null object
cardLast4Digits             641914 non-null object
cardPresent                 641914 non-null bool
creditLimit                 641914 non-null float64
currentBalance              641914 non-null float64
currentExpDate              641914 non-null object
customerId                  641914 non-null object
dateOfLastAddressChange     641914 non-null object
echoBuffer                  0 non-null float64
enteredCVV                  641914 non-null object
expirationDateKeyInMatch    641914 non-null bool
isFraud                     641914 non-null bool
merchantCategoryCode        641914 non-null object
merchantCity                0

There are 641914 instances in the dataset. We have some numerical attributes like 'availableMoney', 'creditLimit', and some categorical attributes like 'merchantCategoryCode', 'merchantName'. We have few attributes which totally have missing values. These attributes are:

- echoBuffer  
- merchantCity
- merchantState
- merchantZip
- posOnPremises
- recurringAuthInd
 

We can drop these columns. Some attributes like 'acqCountry' has around 3913 missing values. first, We need to handle these missing values.

### handeling missing values

We have few options:

- totally drop those attributes from data.
- Drop those records (remove rows where these attributes are missing)
- Set the missing to some values. For numerical attributes, we can set them to the mean/median, and for categorical attributes we can set them to the most frequent category.



In [9]:
df.describe() # Basis statistical summary for numerical attributes.

,availableMoney,creditLimit,currentBalance,echoBuffer,merchantCity,merchantState,merchantZip,posOnPremises,recurringAuthInd,transactionAmount
count,641914.000000,641914.000000,641914.000000,0.0,0.0,0.0,0.0,0.0,0.0,641914.000000
mean,6652.828573,10697.210608,4044.382035,NaN,NaN,NaN,NaN,NaN,NaN,135.162497
std,9227.132275,11460.359133,5945.510224,NaN,NaN,NaN,NaN,NaN,NaN,147.053302
min,-1244.930000,250.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
25%,1114.970000,5000.000000,502.442500,NaN,NaN,NaN,NaN,NaN,NaN,32.320000
50%,3578.165000,7500.000000,2151.860000,NaN,NaN,NaN,NaN,NaN,NaN,85.800000
75%,8169.185000,15000.000000,5005.890000,NaN,NaN,NaN,NaN,NaN,NaN,189.030000
max,50000.000000,50000.000000,47496.500000,NaN,NaN,NaN,NaN,NaN,NaN,1825.250000


Let's look at categorical features. For these features we can count unique value of them. We have some features like date, time which has objcet type, but it does not make sense to count their unique values. So, we are not including them in our categorical feature list below.

In [9]:
cat_col =['acqCountry','cardPresent','expirationDateKeyInMatch','isFraud','merchantCategoryCode',
          'merchantCountryCode','merchantName','posConditionCode','posEntryMode','transactionType']
for c_ in cat_col:
    print('count of unique values of {} is as follows'.format(str(c_)))
    print(df[c_].value_counts())
    print('\n')

count of unique values of acqCountry is as follows
US     632303
MEX      2626
CAN      1870
PR       1202
Name: acqCountry, dtype: int64


count of unique values of cardPresent is as follows
False    340453
True     301461
Name: cardPresent, dtype: int64


count of unique values of expirationDateKeyInMatch is as follows
False    640945
True        969
Name: expirationDateKeyInMatch, dtype: int64


count of unique values of isFraud is as follows
False    630612
True      11302
Name: isFraud, dtype: int64


count of unique values of merchantCategoryCode is as follows
online_retail           161469
fastfood                101196
entertainment            69138
food                     68245
rideshare                50574
online_gifts             33045
hotels                   22879
fuel                     22566
subscriptions            18376
personal care            16917
mobileapps               14614
health                   14344
online_subscriptions     11247
auto                    